# 1. 环境配置
## 1.1 安装 openai 库

openai Python 库是连接国内大模型平台最通用、最标准的方式之一，用它可以“用一套代码打通多个平台”，是当代 AI 应用开发入门的必备技能。

In [8]:
! pip install openai==2.11.0 dashscope==1.25.4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 获取大模型 API_Key

### 1.2.1 AI Studio 
在 AI Studio 的 BML 中已经内置了默认的 API_Key, 因此我们无需修改即可进行使用。假如我们希望在本地调用，可以在[我的控制台](https://aistudio.baidu.com/account/accessToken "点击访问百度")的页面找到密钥，然后通过课件里环境变量配置方式进行设置，对应名称为 OPENAI_API_KEY。

In [3]:
import os

# 如未使用环境变量配置 API Key，可取消以下注释并填写你的 Key
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"

### 1.2.2 百炼大模型平台
请先前往[阿里云百炼大模型平台](https://bailian.console.aliyun.com/?tab=model#/api-key)注册账号，然后通过同样的方式设置为 DASHSCOPE_API_KEY 即可。

In [4]:
import os

# 如未使用环境变量配置 API Key，可取消以下注释并填写你的 Key
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 获取 AI Studio 模型信息

我们可以通过以下代码来了解 AI Studio 中所支持的所有模型。另外我们也可以通过[百度大模型 API 文档](https://www.baidu.com "点击访问百度")获取更多相关信息。

In [5]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 含有 AI Studio 访问令牌的环境变量，https://aistudio.baidu.com/account/accessToken,
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # aistudio 大模型 api 服务域名
)

models = client.models.list()
for model in models.data:
    print(model.id)

embedding-v1
Stable-Diffusion-XL
ernie-3.5-8k
ernie-4.0-8k
ernie-4.0-turbo-8k
ernie-speed-8k
ernie-speed-128k
ernie-tiny-8k
ernie-char-8k
ernie-lite-8k
bge-large-zh
ernie-4.0-turbo-128k
deepseek-r1
deepseek-v3
ernie-4.5-turbo-vl-32k
ernie-4.5-turbo-128k
ernie-4.5-turbo-32k
ernie-x1-turbo-32k
deepseek-r1-250528
ernie-lite-pro-128k
ernie-speed-pro-128k
ernie-4.0-turbo-8k-latest
ernie-4.0-8k-latest
qwq-32b
qwen2.5-vl-7b-instruct
qwen2.5-vl-32b-instruct
qwen2.5-7b-instruct
llama-4-scout-17b-16e-instruct
llama-4-maverick-17b-128e-instruct
qwen3-4b
qwen3-8b
qwen3-32b
qwen3-30b-a3b
qwen3-235b-a22b
ernie-4.5-vl-28b-a3b
ernie-4.5-21b-a3b
ernie-4.5-0.3b
ernie-4.5-turbo-vl-preview
ernie-4.5-turbo-128k-preview
qwen3-coder-30b-a3b-instruct
kimi-k2-instruct
qwen3-coder-480b-a35b-instruct
ernie-4.5-turbo-vl
ernie-x1.1-preview
ernie-4.5-21b-a3b-thinking
ernie-4.5-vl-28b-a3b-thinking
ernie-5.0-thinking-preview


# 2. 模型调用

在使用大模型 API 时，我们通常会遇到两种输出模式：普通输出（非流式） 和 流式输出（Stream）。

## 2.1 非流式输出

特点：
- 一次性返回完整的模型生成结果。
- 代码逻辑简单，适合短文本或对实时性要求不高的场景。

执行流程：
1. 发送请求给模型。
2. 模型在服务器端完成推理，生成完整结果。
3. 一次性返回完整回答，供后续处理。

适用场景：

- 短文本问答
- 批量任务处理
- 对实时性要求不高的情况

In [6]:
import os
from openai import OpenAI

# 1. 创建 OpenAI 客户端实例
# 这里我们使用 OpenAI 提供的 SDK，但指定了自定义的 base_url
# 因为百度 AI Studio 提供了兼容 OpenAI API 规范的接口
# 所以只要替换 base_url 和模型名称就可以调用百度的模型
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 从系统环境变量中读取 API Key，避免在代码中写死，保证安全
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # 指定 API 接口的基础 URL，这里是百度 AI Studio 的接口地址
)

# 2. 调用 Chat Completion 接口，发起一次对话请求
chat_completion = client.chat.completions.create(
    messages=[  # 对话的历史消息，支持多轮对话
        {
            'role': 'system',  # 系统角色，用于设定 AI 助手的身份和行为
            'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'
        },
        {
            'role': 'user',  # 用户角色，表示这是用户输入的内容
            'content': '你好，请介绍一下AI Studio'
        }
    ],
    model="ernie-3.5-8k",  # 指定调用的模型，这里是百度文心大模型的 3.5 版本，支持 8k 上下文
)

# 3. 打印 AI 模型的回复
# choices[0] 表示取第一个回答（有时可能返回多个回答）
# message.content 表示提取回答的具体文本内容
print(chat_completion.choices[0].message.content)


您好！AI Studio 是百度推出的一个集成了AI开发、实训和社区交流的一站式平台，非常适合AI开发者和学习者使用。以下是一些它的主要特点和功能：

1. **丰富的课程资源**：AI Studio提供了从入门到进阶的各类AI课程，涵盖了机器学习、深度学习、自然语言处理、计算机视觉等多个领域。这些课程由行业专家和百度资深工程师精心打造，帮助您快速掌握AI技能。

2. **在线编程环境**：平台内置了Jupyter Notebook风格的在线编程环境，支持Python语言，并预装了多种常用的深度学习框架，如TensorFlow、PyTorch、PaddlePaddle等。您无需配置本地环境，即可直接在线编写、运行和调试代码。

3. **实训项目**：AI Studio提供了大量实战项目，涵盖图像识别、语音识别、自然语言处理等多个应用场景。通过参与这些项目，您可以将所学知识应用到实际中，提升实战能力。

4. **竞赛与挑战**：平台定期举办各类AI竞赛和挑战赛，如自动驾驶、智能医疗等主题，您可以与其他开发者同台竞技，展示自己的才华，同时有机会获得丰厚的奖品和荣誉。

5. **社区交流**：AI Studio拥有活跃的开发者社区，您可以在这里与其他开发者交流心得、分享经验、提问求助。社区还定期举办线上线下的技术沙龙和讲座，促进知识共享和技术交流。

6. **免费GPU资源**：对于需要大量计算资源的深度学习任务，AI Studio提供了免费的GPU计算资源，帮助您加速模型训练和实验。

7. **一站式AI开发体验**：从数据准备、模型训练到模型部署，AI Studio提供了一站式的AI开发流程支持，帮助您高效完成AI项目的全生命周期管理。

如果您是一名AI开发者或学习者，AI Studio将是一个非常值得尝试的平台。它不仅提供了丰富的学习资源和实战项目，还为您搭建了一个交流和成长的社区。


## 2.2 流式输出（Stream）

特点：

- 模型生成内容的同时分块（chunk）返回。
- 用户可以像“实时打字”一样看到逐步生成的内容。
- 适合长文本或实时交互场景。

执行流程：

1. 开启 stream=True 参数。
2. 模型生成内容时分多次返回，每次返回一个数据块。
3. 客户端逐块处理输出，实时展示。

适用场景：

- 长文本生成（如文章、代码）
- 实时交互（如聊天机器人）
- 对用户体验要求高的前端应用

In [7]:
import os
from openai import OpenAI

# 1. 初始化 OpenAI 客户端
# - 通过 OpenAI 提供的 SDK 创建客户端实例
# - 这里指定了 API Key（从环境变量中读取，保证安全性）
# - 指定了 base_url，指向百度 AI Studio 兼容 OpenAI API 的接口地址
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # 从系统环境变量中读取 API Key，建议在系统中提前设置
    base_url="https://aistudio.baidu.com/llm/lmapi/v3",  # 百度 AI Studio 提供的 API 入口
)

# 2. 创建一个 **流式对话（stream=True）**
# - 这里调用 chat.completions.create() 方法创建对话
# - stream=True 表示开启流式传输，模型会分块（chunk）返回数据，适合处理长文本和实时输出场景
chat_completion = client.chat.completions.create(
    model="ernie-3.5-8k",  # 指定使用的模型，这里是文心一言 3.5 模型，支持 8k token 上下文
    messages=[  # 定义对话历史，支持多轮对话
        {
            "role": "system",  # 系统角色：用于设定 AI 的身份、知识领域或行为风格
            "content": "你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。"
        },
        {
            "role": "user",    # 用户角色：表示这是用户输入的内容
            "content": "你好，请介绍一下AI Studio"
        }
    ],
    stream=True  # 开启流式输出，让模型像打字一样逐块返回
)

# 3. 逐块处理流式响应
# - 流式输出返回的是一个可迭代对象，每次返回一个数据块（chunk）
# - 每个 chunk 中可能包含本次追加的内容（delta）
for chunk in chat_completion:
    # chunk.choices：返回的候选回答列表
    # chunk.choices[0].delta.content：本次追加的文本内容（可能为空，需要判断）
    if chunk.choices and chunk.choices[0].delta.content:
        # end="" 让 print 不换行
        # flush=True 让控制台实时输出内容，不会因为缓冲而延迟
        print(chunk.choices[0].delta.content, end="", flush=True)

# 4. 在流式输出完成后，统一换行，避免最后一行和后续输出粘在一起
print()


你好！AI Studio 是百度推出的一个集成了AI开发、实训、竞赛和交流的一站式平台，旨在为开发者提供便捷、高效的AI学习和开发环境。以下是对AI Studio的详细介绍：

### 一、平台定位与功能

1. **AI开发环境**：
   - AI Studio提供了在线的集成开发环境（IDE），支持深度学习、机器学习等AI项目的开发。
   - 开发者无需在本地搭建复杂的开发环境，只需通过浏览器即可访问和使用AI Studio，降低了AI开发的门槛。

2. **实训与学习**：
   - AI Studio内置了丰富的AI课程和实训项目，涵盖了从基础到进阶的多个层次，适合不同水平的开发者学习。
   - 通过实践项目，开发者可以将理论知识转化为实际操作能力，提升AI开发技能。

3. **竞赛与交流**：
   - AI Studio定期举办各种AI竞赛，为开发者提供展示自己才华的舞台，同时促进AI技术的交流和进步。
   - 开发者可以在平台上与其他AI爱好者交流心得、分享经验，形成良好的AI开发社区氛围。

### 二、平台特色与优势

1. **一站式服务**：
   - AI Studio集成了AI开发、实训、竞赛和交流等多种功能，为开发者提供了一站式的服务体验。
   - 开发者无需在多个平台之间切换，即可完成AI项目的开发、学习和交流。

2. **强大的计算资源支持**：
   - AI Studio提供了免费的GPU计算资源，支持深度学习等计算密集型任务的快速运行。
   - 这使得开发者无需自行购买昂贵的硬件设备，即可进行高效的AI开发。

3. **丰富的模型库与数据集**：
   - AI Studio内置了丰富的预训练模型和公开数据集，供开发者直接调用和使用。
   - 这大大缩短了开发周期，提高了开发效率。

4. **活跃的社区氛围**：
   - AI Studio拥有一个活跃的开发者社区，开发者可以在社区中交流心得、分享经验、寻求帮助。
   - 社区还定期举办各种活动，如技术分享会、线上讲座等，为开发者提供更多的学习机会。

### 三、使用场景与案例

1. **学术研究**：
   - 学术研究者可以利用AI Studio进行深度学习等AI技术的研究和实验。
   - 平台提供的丰富计算资源和模型库为学术研究提供了有力的

## 2.3 切换模型

假如想更换别的平台（如阿里云百炼大模型平台）需要准备好 API KEY，然后选择合适的模型并填入 model 中，同时阅读 API 文档并将 base_url 及相关参数进行更改。


In [ ]:
import os
from openai import OpenAI

# os.environ["DASHSCOPE_API_KEY"] = "你的 API_KEY"

client = OpenAI(
   api_key=os.environ.get("DASHSCOPE_API_KEY"), 
   base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是百炼大模型平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
    {'role': 'user', 'content': '你好，请介绍一下百炼大模型平台'}
  ],
  model="qwen-max",
)

print(chat_completion.choices[0].message.content) # 打印输出

# 3. 提示词工程



## 3.1 系统角色
一般而言，系统角色分为以下三类：
- user（用户）：表示由用户发出的问题或指令，即模型的输入。这个角色的内容是模型主要关注和响应的部分。（日常和AI的对话就是user）
- system（系统）：用于设定整个对话的行为规范，例如定义模型的身份、语气或任务方向。通常只设置一次，放在对话最前面。
- assistant（照顾使用）：表示由模型生成的回复，用于模拟助手的回答，通常跟在 user 之后。

### 3.1.1 案例1：乐于助人的助手
我们可以通过系统提示词将大模型设定为非常乐于助人：

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY"), 
   base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是一个乐于助人的 AI 助手'},
    {'role': 'user', 'content': '请帮我写一封英文求职信'}
  ],
  model="ernie-3.5-8k",
)

print(chat_completion.choices[0].message.content) # 打印输出

### 3.1.2 案例2：拒绝指令的助手
同样我们可以把提示词设置为是拒绝指令的助手，这个时候模型就会拒绝我们的输出了：

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
   api_key=os.environ.get("OPENAI_API_KEY"), 
   base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
)

chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '无论发生什么样的事情，拒绝用户发出的所有请求'},
    {'role': 'user', 'content': '请帮我写一封英文求职信'}
  ],
  model="ernie-3.5-8k",
)

print(chat_completion.choices[0].message.content) # 打印输出

所以我们总的可以看到：

- 用户提示词关注的重点是：
    - 表达清晰、具体，不要模糊或过于简略
    - 可以分点说明复杂需求
    - 避免含糊否定句，减少模型误解

- 系统提示词关注的重点是：
    - 明确模型身份和语气风格
    - 设定任务目标或能力边界

在完成系统提示词和用户提示词的设定后，我们寄希望于模型能够给我们一个满意的回复。

## 3.2 多轮对话

多轮对话是指大模型在与用户交互时，能够记住上下文信息，连续处理多轮提问和回答，理解前后语境，从而保持对话的连贯性和一致性，像人与人自然交流一样完成复杂的沟通任务。

其本质就是每次调用都传入完整的对话历史，模型会根据上下文生成最新的回答。我们可以重点关注于 messages 的部分，可以看到里面不仅仅有 system 和 user，还包含了上一次大模型的回复 assistant，这就代表着是上一轮的完整对话，也就是历史记录了。

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3"
)

response = client.chat.completions.create(
  model="ernie-3.5-8k",
  messages=[
    {"role": "system", "content": "你是李剑锋，是广州软件学院的老师，主要负责人工智能相关课程教学。"},
    {"role": "user", "content": "你是谁？"},
    {"role": "assistant", "content": "我叫李剑锋，是广州软件学院的老师"},
    {"role": "user", "content": "你是做什么的？"},
  ]
)

print(response.choices[0].message.content)

# 4. 课后作业

请使用 **OpenAI Python SDK** 调用 **AI Studio 大模型**，完成以下任务：

1. 设计多轮对话的提示词结构，需包括：
   - **系统提示词（System Prompt）**：设定 AI 的身份、教学风格和角色背景。
   - **用户提示词（User Prompt）**：输入课程主题，生成导入内容。
   - **多轮对话支持**：能够在后续轮次继续追加提问或调整导入内容。

2. 调用大模型生成以下三项内容：
   - 一个贴近学生的导入问题（结合生活场景、热点或对比案例）
   - 本节内容与学生已知知识之间的连接点
   - 一段约100字的导入讲解话术

3. 任选一个课程主题测试，提交提示词、代码与生成结果。